This program compares the ZCTA-tract crosswalk with the EJ40 database to generate a percentage of tracts within a zip code that are identified EJ communities. 

This is not a perfect approximation for the percentage of people in a zip code living in EJ communities; while census tracts are typically similar in population, they can vary from between 1200 and 8000 people, and some census tracts are split across zip codes.

In [17]:
import pandas

crosswalk = r'/Users/robert/Downloads/_Python/EJ40/ZCTA2tract10.csv'
ej40table = r'/Users/robert/Downloads/_Python/EJ40/communities-2022-05-31-1915GMT.csv'

outpath = r'/Users/robert/Downloads/_Python/EJ40/ZCTA_EJ.csv'

lookup = pandas.read_csv(crosswalk)
ej = pandas.read_csv(ej40table)

In [19]:
zcta_name = 'ZCTA5'
tract_geoid_name = 'GEOID'

#uncomment these lines if using 2020 ZCTA lookup
#zcta_name = 'NAMELSAD_ZCTA5_20'
#tract_geoid_name = 'GEOID_TRACT_20'

#remove census tracts which don't have an associated zip code (eg, national parks)
lookup = lookup.dropna()

#keep only the columns we need: ZCTA and tract for the lookup, tract ID and if it's EJ from the EJ40 table
lookup = lookup[[zcta_name, tract_geoid_name]]
ej = ej[['Census tract ID','Identified as disadvantaged']]

In [27]:
#format the zip codes nicely and set census tract IDs as the indices for the ej table 

#Only run this block ONCE

#Uncomment this if using 2020 tracts
#for i, row in lookup.iterrows():
#    lookup.at[i, zcta_name] = row[0][6:]
    
ej.set_index('Census tract ID',inplace=True)

In [26]:
lookup

,ZCTA5,GEOID
0,601,72001956300
1,601,72001956400
2,601,72001956500
3,601,72001956600
4,601,72001956700
...,...,...
148892,99923,2198000300
148893,99925,2198000200
148894,99926,2198940100
148895,99927,2198000100


In [28]:
joined = pandas.DataFrame(columns=['zip', 'num_tracts', 'percent_ej'])
tracts_not_found = []

prev_zip = 0
for i, row in lookup.iterrows():
    this_zip = row[0]
    try:
        ej_int = int(ej.loc[row[1]]) * 100 #100 if the tract is an EJ community, 0 otherwise
        if this_zip == prev_zip:
            current_ind = len(joined) - 1
            num_tracts = joined.at[current_ind, 'num_tracts']
            print()
            joined.at[current_ind, 'num_tracts'] = num_tracts + 1
            percent_calc = ((joined.at[current_ind, 'percent_ej'] * num_tracts) + ej_int) / (num_tracts + 1)
            joined.at[current_ind, 'percent_ej'] = percent_calc
        else:
            joined.loc[len(joined)] = [this_zip, 1, ej_int]
        prev_zip = this_zip
    except:
        tracts_not_found.append(row[1])

In [30]:
joined.to_csv(outpath)

In [29]:
import numpy
txtname = r'/Users/robert/Downloads/_Python/EJ40/missing_tracts'
len(tracts_not_found)

0